|평가문항|상세기준|
|:------|:---|
|1. 모델과 데이터를 정상적으로 불러오고, 작동하는 것을 확인하였다. |klue/bert-base를 NSMC 데이터셋으로 fine-tuning하여, 모델이 정상적으로 작동하는 것을 확인하였다.|
|2. Preprocessing을 개선하고, fine-tuning을 통해 모델의 성능을 개선시켰다. | Validation accuracy를 90%이상으로 개선하였다.|
|3. 모델학습에 Bucketing을 성공적으로 적용하고, 그 결과를 비교분석하였다. | Bucketing Task를 수행하여 fine-tuning 시 연산 속도와 모델 성능가느이 trade-off 관계가 발생하는지 여부를 확인하고, 분서갛ㄴ 결과를 제시하였다.|

### Step 0. 패키지 호출

In [1]:
# !pip install transformers==4.4.2

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import transformers
import datasets
from datasets import load_dataset, DatasetDict, Dataset
import tensorflow_datasets as tfds

from transformers import AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, DataCollatorForLanguageModeling
# from transformers import AutoModelForSequenceClassificiation

from sklearn.metrics import accuracy_score

In [3]:
!nvidia-smi

Fri Sep 15 15:46:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Step 1. NSMC 데이터 분석 및 Huggingface dataset 구성

데이터셋은 깃허브에서 다운받거나, Huggingface datasets에서 가져올 수 있습니다. 앞에서 배운 방법들을 활용해봅시다!

#### 커스텀 데이터셋 만들기

In [4]:
huggingface_dataset = load_dataset('nsmc')
print(huggingface_dataset)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/6.33M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset nsmc downloaded and prepared to /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [5]:
train = huggingface_dataset['train']
cols = train.column_names
cols

['id', 'document', 'label']

In [6]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




In [7]:
# Tensorflow dataset 구조를 python dict 형식으로 변경
# Dataset이 train, validation, test로 나뉘도록 구성

train_valid = huggingface_dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_valid['train']
val_dataset = train_valid['test']
test_dataset = huggingface_dataset['test']

# dataframe 데이터를 dict 내부에 list로 변경
train_dataset = train_dataset.to_dict('list')
val_dataset = val_dataset.to_dict('list')
test_dataset = test_dataset.to_dict('list')

# Huggingface dataset
tf_train_dataset = Dataset.from_dict(train_dataset)
tf_val_dataset = Dataset.from_dict(val_dataset)
tf_test_dataset = Dataset.from_dict(test_dataset)

### Step 2. klue/bert-base model 및 tokenizer 불러오기

In [8]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels = 2)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [9]:
def transform(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,
        )

# Q. sample 5개를 불러와 tokenizer의 결과를 출력해주세요
hf_dataset = huggingface_dataset.map(transform, batched=True)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [10]:
print(hf_dataset)

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
        num_rows: 50000
    })
})


In [11]:
train_valid = hf_dataset['train'].train_test_split(test_size=0.2)
train_dataset = train_valid['train']
val_dataset = train_valid['test']
test_dataset = hf_dataset['test']

### Step 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해보기

In [12]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 4,   # 각 device 당 batch size
    per_device_eval_batch_size = 4,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

In [13]:
from datasets import load_metric
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

In [14]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [15]:
trainer = Trainer(
    model=huggingface_model,        # 학습시킬 model
    args=training_arguments,        # TrainingArguments을 통해 설정한 arguments
    data_collator = DataCollatorWithPadding(tokenizer=huggingface_tokenizer),
    train_dataset=train_dataset,    # training dataset
    eval_dataset=val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()
print("슝~")

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running training *****
  Num examples = 120000
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 90000


ResourceExhaustedError: /aiffel/aiffel/transformers/runs/Sep15_15-47-25_whd5h8m3qk4hvypkwp4qxrbwl-85d8c9bf7-vfttp; No space left on device

In [ ]:
!nvidia-smi

In [ ]:
trainer.evaluate(test_dataset)

print("완료")

### Step 4. Fine-tuning을 통하여 모델 성능(accuracy)향상시키기

In [ ]:
training_arguments2 = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 4,   # 각 device 당 batch size
    per_device_eval_batch_size = 4,    # evaluation 시에 batch size
    num_train_epochs = 1,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

In [ ]:
trainer2 = Trainer(
    model=huggingface_model,        # 학습시킬 model
    args=training_arguments2,        # TrainingArguments을 통해 설정한 arguments
    data_collator = DataCollatorWithPadding(tokenizer=huggingface_tokenizer),
    train_dataset=train_dataset,    # training dataset
    eval_dataset=val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
    group_by_length = True
)
trainer2.train()
print("슝~")

In [ ]:
trainer2.evaluate(test_dataset)
print("완료")

### Step 5. Bucketing을 작용하여 학습시키고, Step 4의 결과와의 비교

In [ ]:
training_arguments3 = TrainingArguments(
    output_dir,                        # output이 저장될 경로
    evaluation_strategy="epoch",       #evaluation하는 빈도
    learning_rate = 2e-5,              #learning_rate
    per_device_train_batch_size = 4,   # 각 device 당 batch size
    per_device_eval_batch_size = 4,    # evaluation 시에 batch size
    num_train_epochs = 1,              # train 시킬 총 epochs
    warmup_steps=1000, 
    weight_decay = 0.01,               # weight decay
)

In [ ]:
trainer3 = Trainer(
    model=huggingface_model,        # 학습시킬 model
    args=training_arguments3,        # TrainingArguments을 통해 설정한 arguments
    data_collator = DataCollatorWithPadding(tokenizer=huggingface_tokenizer),
    train_dataset=train_dataset,    # training dataset
    eval_dataset=val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
    group_by_length = True
)
trainer3.train()
print("슝~")

In [ ]:
trainer3.evaluate(test_dataset)
print("완료")